In [1]:
import pandas as pd
import numpy as np
import data_preprocessing as dp
import mdp
import Replay_Buffer
import torch as T

### load training data

In [2]:
# load data: DR_USA_Roundabout_FT_train.csv

# Roundabout data google drive path: "https://drive.google.com/file/d/1Ls92QosBDIyoDcEudSVGukP68pw1FRXW/view?usp=sharing"
# please download it to your local computer

url_FT_train = 'INTERACTION/DR_USA_Roundabout_FT_train.csv'
df_train = pd.read_csv(url_FT_train)

# look at the top 30 rows of data
df_train.head(30)

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width
0,1.0,1,1,100,pedestrian/bicycle,1044.509,1012.175,0.103,-1.005,NaN,NaN,NaN
1,1.0,2,1,100,pedestrian/bicycle,1043.840,1013.348,-0.101,-1.362,NaN,NaN,NaN
2,1.0,3,1,100,pedestrian/bicycle,1007.638,1022.038,0.687,0.787,NaN,NaN,NaN
3,1.0,4,1,100,car,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86
4,1.0,5,1,100,car,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90
5,1.0,6,1,100,car,1004.449,984.315,1.823,-0.369,-0.200,4.93,1.97
6,1.0,7,1,100,car,1013.407,987.769,4.564,-3.386,-0.638,4.83,1.93
7,1.0,8,1,100,car,1048.917,980.407,-0.894,0.591,2.557,5.13,1.96
8,1.0,9,1,100,car,993.728,985.780,1.184,0.012,0.010,4.54,1.81
9,1.0,10,1,100,car,981.026,985.788,4.411,0.197,0.045,4.55,1.78


### 1. create unique_id
### 2. change length, width and psi_rad from NaN to 0 for pedestrain/bicycle
### 3. change agent type to number for state vector (pedestrian/bicycle = -1; car = 1)

In [3]:
# add a new label unique_id using case_id and track_id
df_train['unique_id'] = df_train['case_id'].astype(int) * 100+ df_train['track_id']

# change length, width and psi_rad from NaN to 0 for pedestrain/bicycle
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'length'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'width'] = 0
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'psi_rad'] = 0


### important!!!
# agent_type pedestrain/bicycle = -1
# agent_type car = 1
df_train.loc[df_train['agent_type'] == 'pedestrian/bicycle', 'agent_type'] = -1
df_train.loc[df_train['agent_type'] == 'car', 'agent_type'] = 1
df_train

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
0,1.0,1,1,100,-1,1044.509,1012.175,0.103,-1.005,0.000,0.00,0.00,101
1,1.0,2,1,100,-1,1043.840,1013.348,-0.101,-1.362,0.000,0.00,0.00,102
2,1.0,3,1,100,-1,1007.638,1022.038,0.687,0.787,0.000,0.00,0.00,103
3,1.0,4,1,100,1,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86,104
4,1.0,5,1,100,1,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4243282,9811.0,8,40,4000,1,1022.107,985.137,7.470,1.627,0.215,4.19,1.87,981108
4243283,9811.0,9,40,4000,1,996.468,986.201,7.374,-0.178,-0.024,4.64,1.90,981109
4243284,9811.0,10,40,4000,1,1019.894,1009.985,-7.780,-0.610,-3.063,4.72,1.87,981110
4243285,9811.0,11,40,4000,1,1028.592,1022.465,-1.619,-1.729,-2.324,4.49,1.83,981111


### use 10 cases (case1 to case10) for demonstrating and testing

In [4]:
# use case 1 to case 100 as a toy test dataframe
number_of_cases = 2

df_train_small = df_train[df_train['case_id'] <= number_of_cases]
df_train_small 

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width,unique_id
0,1.0,1,1,100,-1,1044.509,1012.175,0.103,-1.005,0.000,0.00,0.00,101
1,1.0,2,1,100,-1,1043.840,1013.348,-0.101,-1.362,0.000,0.00,0.00,102
2,1.0,3,1,100,-1,1007.638,1022.038,0.687,0.787,0.000,0.00,0.00,103
3,1.0,4,1,100,1,1036.888,976.415,-1.063,1.980,2.064,5.20,1.86,104
4,1.0,5,1,100,1,1038.206,993.477,-2.343,5.161,1.997,4.68,1.90,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,2.0,8,40,4000,1,1008.481,993.097,3.799,-4.802,-0.901,4.80,1.90,208
878,2.0,9,40,4000,1,1044.608,984.263,-0.540,0.413,2.489,4.48,1.80,209
879,2.0,10,40,4000,1,990.708,985.918,4.940,0.168,0.034,4.61,1.86,210
880,2.0,11,40,4000,1,1036.495,974.560,-2.085,2.679,2.232,4.71,1.89,211


In [5]:
print('--df--')
print('Total number of cases: ', df_train['case_id'].nunique())
print('Total number of agents: ', df_train.unique_id.nunique())
print('--df_small--')
print('Total number of cases: ', df_train_small['case_id'].nunique())
print('Total number of agents: ', df_train_small.unique_id.nunique())

--df--
Total number of cases:  9811
Total number of agents:  129019
--df_small--
Total number of cases:  2
Total number of agents:  27


### create uniqueTracks (a dictionary strore every unique agents (car, bike, or pedestrain)

In [6]:
# load uniqueTracks data
# uniqueTracks is a dictionary strore each unique object
uniqueTracks = dp.read_uniqueTracks(df_train_small)
uniqueTracks

{101: <data_preprocessing.uniqueTrack at 0x12a3e5cd0>,
 102: <data_preprocessing.uniqueTrack at 0x12a3e7c80>,
 103: <data_preprocessing.uniqueTrack at 0x12a3e7860>,
 104: <data_preprocessing.uniqueTrack at 0x12a3e59d0>,
 105: <data_preprocessing.uniqueTrack at 0x12a41e1e0>,
 106: <data_preprocessing.uniqueTrack at 0x12a41ef90>,
 107: <data_preprocessing.uniqueTrack at 0x12a41efc0>,
 108: <data_preprocessing.uniqueTrack at 0x12a41e210>,
 109: <data_preprocessing.uniqueTrack at 0x12a41e270>,
 110: <data_preprocessing.uniqueTrack at 0x12a41e240>,
 111: <data_preprocessing.uniqueTrack at 0x12a3e7830>,
 112: <data_preprocessing.uniqueTrack at 0x12a41c440>,
 113: <data_preprocessing.uniqueTrack at 0x12a41c4a0>,
 114: <data_preprocessing.uniqueTrack at 0x129d53fb0>,
 115: <data_preprocessing.uniqueTrack at 0x12a3e6900>,
 201: <data_preprocessing.uniqueTrack at 0x12a3e6960>,
 202: <data_preprocessing.uniqueTrack at 0x12a41c410>,
 203: <data_preprocessing.uniqueTrack at 0x12a41c320>,
 204: <dat

### information stored in the object

In [7]:
# take a look at the first object
object_1 = uniqueTracks[101]
print(object_1)

uniqueTrack: case_id=1.0, track_id=1,                 unique_id=101, agent_type=-1,                 length=0.0, width=0.0
    100: {'agent_type': -1, 'x': 1044.509, 'y': 1012.175, 'vx': 0.103, 'vy': -1.005, 'psi_rad': 0.0, 'length': 0.0, 'width': 0.0, 'unique_id': 101}
    200: {'agent_type': -1, 'x': 1044.52, 'y': 1012.075, 'vx': 0.116, 'vy': -1.0, 'psi_rad': 0.0, 'length': 0.0, 'width': 0.0, 'unique_id': 101}
    300: {'agent_type': -1, 'x': 1044.533, 'y': 1011.974, 'vx': 0.139, 'vy': -1.012, 'psi_rad': 0.0, 'length': 0.0, 'width': 0.0, 'unique_id': 101}
    400: {'agent_type': -1, 'x': 1044.548, 'y': 1011.872, 'vx': 0.169, 'vy': -1.042, 'psi_rad': 0.0, 'length': 0.0, 'width': 0.0, 'unique_id': 101}
    500: {'agent_type': -1, 'x': 1044.567, 'y': 1011.765, 'vx': 0.201, 'vy': -1.088, 'psi_rad': 0.0, 'length': 0.0, 'width': 0.0, 'unique_id': 101}
    600: {'agent_type': -1, 'x': 1044.588, 'y': 1011.654, 'vx': 0.233, 'vy': -1.143, 'psi_rad': 0.0, 'length': 0.0, 'width': 0.0, 'unique_id'

In [8]:
print('Object type:', object_1.agent_type)
print('Case ID:', object_1.case_id)
print('Object length:', object_1.length)
print('Record once every 0.1 second, total recording time is 4 seconds-> the length of data:', len(object_1.motionState))
print('motion state at 1st 0.1 second', object_1.motionState[100])
print('motion state at 2nd 0.1 second', object_1.motionState[200])
print()

Object type: -1
Case ID: 1.0
Object length: 0.0
Record once every 0.1 second, total recording time is 4 seconds-> the length of data: 40
motion state at 1st 0.1 second {'agent_type': -1, 'x': 1044.509, 'y': 1012.175, 'vx': 0.103, 'vy': -1.005, 'psi_rad': 0.0, 'length': 0.0, 'width': 0.0, 'unique_id': 101}
motion state at 2nd 0.1 second {'agent_type': -1, 'x': 1044.52, 'y': 1012.075, 'vx': 0.116, 'vy': -1.0, 'psi_rad': 0.0, 'length': 0.0, 'width': 0.0, 'unique_id': 101}



## look at a car agent

In [9]:
object_2 = uniqueTracks[104]
print('Object type:', object_2.agent_type)
print('Object length:', object_2.length)
print('Record once every 0.1 second, total recording time is 4 seconds-> the length of data:', len(object_2.motionState))
print('motion state at 1st 0.1 second', object_2.motionState[100])
print('motion state at 2nd 0.1 second', object_2.motionState[200])

Object type: 1
Object length: 5.2
Record once every 0.1 second, total recording time is 4 seconds-> the length of data: 40
motion state at 1st 0.1 second {'agent_type': 1, 'x': 1036.888, 'y': 976.415, 'vx': -1.063, 'vy': 1.98, 'psi_rad': 2.064, 'length': 5.2, 'width': 1.86, 'unique_id': 104}
motion state at 2nd 0.1 second {'agent_type': 1, 'x': 1036.793, 'y': 976.619, 'vx': -1.047, 'vy': 1.985, 'psi_rad': 2.056, 'length': 5.2, 'width': 1.86, 'unique_id': 104}


### get other/interative vehicles in the same case and in the same time frame 

In [10]:
# case 1: the ego is in the current time frame
ego_id = 104
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other agents:", other_agents)
print("interactive agens:", interactive_agents)

other agents: [101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113]
interactive agens: [108, 105, 111]


In [11]:
# case 2: the ego is NOT in the current time frame
ego_id = 114
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
print("other agents:", other_agents)
print("interactive agens:", interactive_agents)

other agents: []
interactive agens: []


### get MDP tuple (State, Action, Reware, Next State, the last episode)
### * if interactive_agents is empty or ego is not a car, then mdp_tuple will return None

#### case 1: the ego is in the current time frame, then get_mdp_tuple will return mdp_tuple

In [12]:
v_max = max(df_train_small['vx']**2 + df_train_small['vy']**2) ** 0.5

In [19]:
ego_id = 104
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
mdp_tuple = mdp.get_mdp_tuple(ego_id, interactive_agents, current_time, uniqueTracks, v_max)
if not mdp_tuple: # if interactive_agents is empty or ego is not a car, then mdp_tuple is None
    print(mdp_tuple)
else:
    s, a, r, s_next, the_last_episode = mdp_tuple
    print('s: ', s)
    print('a: ', a)
    print('r: ', r)
    print('s_next: ', s_next)
    print('the_last_episode: ', the_last_episode)

# s = [ego_id, int_1_id, int_2_id, int_3_id, ego_features, int_1_features, int_2_features, int_3_features]
# ego_features = [ego_x, ego_y, ego_vx, ego_vy, ego_psi_rad, ego_pred_x, ego_pred_y]
# interaction_features = [agent_type, agent_x, agent_y, agent_vx, agent_vy, agent_psi_rad]

# a = [vx, vy] at next timestep
# r = TBD
# s_next = [ego_id, int_1_id, int_2_id, int_3_id, ego_features, int_1_features, int_2_features, int_3_features] at next timestep
# the_last_episode = True if it's the last episode
print(len(s))
print(len(s_next))


s:  [104, 108, 105, 111, 1036.888, 976.415, -1.063, 1.98, 2.064, 5.2, 1.86, 104, 1035.8240141897138, 978.3944704330981, 1, 1048.917, 980.407, -0.894, 0.591, 2.557, 5.13, 1.96, 108, 1, 1038.206, 993.477, -2.343, 5.161, 1.997, 4.68, 1.9, 105, 1, 1055.975, 975.473, -1.121, 0.764, 2.543, 4.74, 1.86, 111]
a:  [-1.047, 1.985, 2.056]
r:  0.09974544347140994
s_next:  [104, 108, 105, 111, 1036.793, 976.619, -1.047, 1.985, 2.056, 5.2, 1.86, 104, 1035.7463308137817, 978.6041744544552, 1, 1048.827, 980.465, -0.92, 0.608, 2.558, 5.13, 1.96, 108, 1, 1037.975, 993.996, -2.341, 5.201, 1.994, 4.68, 1.9, 105, 1, 1055.864, 975.551, -1.13, 0.773, 2.542, 4.74, 1.86, 111]
the_last_episode:  False
41
41


/Users/kycheng/Desktop/5_Winter2024/1_EECS 545/project/github/EECS-545-final-project/reward.py:162: RuntimeWarning: invalid value encountered in arccos
  alpha = np.arccos((distance_next**2 + distance_closest**2 - distance_next_closest**2)/(2*distance_next*distance_closest))
/Users/kycheng/Desktop/5_Winter2024/1_EECS 545/project/github/EECS-545-final-project/reward.py:119: RuntimeWarning: invalid value encountered in arccos
  if rotated_rectangles_intersect(((ego_x, ego_y), 0.95 * ego_length, 0.95 * ego_width, np.arccos(ego_psi)),
/Users/kycheng/Desktop/5_Winter2024/1_EECS 545/project/github/EECS-545-final-project/reward.py:120: RuntimeWarning: invalid value encountered in arccos
  ((interactive_x, interactive_y), 0.95 * interactive_length, 0.95 * interactive_width, np.arccos(interactive_psi))):


#### case 2: the ego is NOT in the current time frame, then get_mdp_tuple will return None

In [14]:
# case 2: the ego is NOT in the current time frame
ego_id = 114
current_time = 100

# get other vehicles with same case and time
other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
mdp_tuple = mdp.get_mdp_tuple(ego_id, interactive_agents, current_time, uniqueTracks, v_max)
if not mdp_tuple: # if interactive_agents is empty or ego is not a car, then mdp_tuple is None
    print(mdp_tuple)
else:
    s, a, r, s_next, the_last_episode = mdp_tuple
    print('s: ', s)
    print('a: ', a)
    print('r: ', r)
    print('s_next: ', s_next)
    print('the_last_episode: ', the_last_episode)


None


#### test for final episode

In [15]:
ego_id = 104
current_time = 3900

other_agents, interactive_agents = mdp.get_other_agents_unique_id(ego_id, current_time, uniqueTracks)
mdp_tuple = mdp.get_mdp_tuple(ego_id, interactive_agents, current_time, uniqueTracks, v_max)
if not mdp_tuple: # if interactive_agents is empty or ego is not a car, then mdp_tuple is None
    print(mdp_tuple)
else:
    s, a, r, s_next, the_last_episode = mdp_tuple

the_last_episode

True

In [16]:
#Set Device
device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')

In [17]:
#Initialize the buffer
replay_buffer = Replay_Buffer.ReplayBuffer(28, 2, device, 100000)

In [18]:
# Loop to add each transition to the replay buffer
# Once all added we can save and then load at time of training
for unique_id,group in df_train_small.groupby('unique_id'):
    timestamps = group['timestamp_ms'].unique()
    num_timestamps = len(timestamps)
    for timestamp in timestamps[:num_timestamps - 1]:
        frame = group[group['timestamp_ms'] == timestamp]
        ego_id = unique_id
        _, interactive_agents = mdp.get_other_agents_unique_id(ego_id, timestamp, uniqueTracks)

        mdp_tuple = mdp.get_mdp_tuple(ego_id, interactive_agents, timestamp, uniqueTracks)
        if not not mdp_tuple:
            s, a, r, s_next, the_last_episode = mdp_tuple
        
        replay_buffer.add(s, a, s_next, r, the_last_episode)
        i=i+1

print(i) #Total transitions we should have !!!

TypeError: get_mdp_tuple() missing 1 required positional argument: 'v_max'

In [ ]:
#Sample the Buffer
test = replay_buffer.sample(2)
test

In [ ]:
#Save the Buffer
replay_buffer.save("buffers/DR_USA_Roundabout_FT")

In [ ]:
#Load the Buffer
replay_buffer_load = Replay_Buffer.ReplayBuffer(28, 2, device, 100000)
replay_buffer_load.load(f"buffers/DR_USA_Roundabout_FT")

In [ ]:
replay_buffer_load.sample(2)